In [1]:
import geojson
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dfs = pd.DataFrame({})

for i in range(4,9):
    #Weekdays
    df = pd.read_csv('./csv/sms-call-internet-mi-2013-11-0{}.csv'.format(i), parse_dates=['time'], infer_datetime_format=True)
    dfs = dfs.append(df)


## AGGREGATE SMS & CALLS ##
#dfs['sms'] = dfs['smsin'] + dfs['smsout']
#dfs['calls'] = dfs['callin'] + dfs['callout']
#dfsn['sms'] = dfsn['smsin'] + dfsn['smsout']
#dfsn['calls'] = dfsn['callin'] + dfsn['callout']

dfs = dfs.fillna(0)

In [3]:
#dfs.head
#del dfs["Unnamed: 0"]
dfs.head()

,Unnamed: 0,cellid,time,countrycode,smsin,smsout,callin,callout,internet
0,0,1,2013-11-04 00:00:00,0,0.108039,0.000000,0.000000,0.000000,0.000000
1,1,1,2013-11-04 00:00:00,39,0.026137,0.030875,0.026137,0.055225,9.260190
2,2,1,2013-11-04 00:10:00,39,0.187075,0.106876,0.027300,0.000000,9.215273
3,3,1,2013-11-04 00:20:00,39,0.027925,0.027925,0.001787,0.054601,8.669007
4,4,1,2013-11-04 00:30:00,39,0.190564,0.106876,0.000000,0.026137,7.811099


In [4]:
cells = 10000+1
smsipeak_wd = []
sintime=[]
smsopeak_wd = []
callipeak_wd = []
callopeak_wd = []
intpeak_wd = []

#smsipeak_tr = []
#smsopeak_tr = []
#callipeak_tr = []
#callopeak_tr = []
#intpeak_tr = []

for i in range(1, cells):
    
    #Working hours counters
    tmpsi=[0]
    tmpso=[0]
    tmpci=[0]
    tmpco=[0]
    tmpin=[0]

    #Home counters
    tmpsi2=[0]
    tmpso2=[0]
    tmpci2=[0]
    tmpco2=[0]
    tmpin2=[0]

    #Transport counters
    #trsi=[0]
    #trso=[0]
    #trci=[0]
    #trco=[0]
    #trin=[0]
    
    #Weekdays
    #Len(xxxdata) == len(stime)
    sindata = (dfs[dfs.cellid==i]['smsin']).to_list()
    soutdata = dfs[dfs.cellid==i]['smsout'].to_list()
    cindata = dfs[dfs.cellid==i]['callin'].to_list()
    coutdata = dfs[dfs.cellid==i]['callout'].to_list()
    intdata = dfs[dfs.cellid==i]['internet'].to_list()
    
    stime=(dfs[dfs.cellid==i]['time']).to_list()
    
    for j in range(0, len(stime)):
        #if(stime[j].hour+stime[j].minute/60 >= 7 and stime[j].hour+stime[j].minute/60 <= 9): #Transport (green)
        #    trsi.append(sindata[j])
        #    trso.append(soutdata[j])
        #    trci.append(cindata[j])
        #    trco.append(coutdata[j])
        #    trin.append(intdata[j])
        if(stime[j].hour+stime[j].minute/60 >= 10.5 and stime[j].hour+stime[j].minute/60 <= 12): #Working hours (red)
            tmpsi.append(sindata[j])
            tmpso.append(soutdata[j])
            tmpci.append(cindata[j])
            tmpco.append(coutdata[j])
            tmpin.append(intdata[j])
        if(stime[j].hour+stime[j].minute/60 >= 14 and stime[j].hour+stime[j].minute/60 <= 22): #Home hours (blue)
            tmpsi2.append(sindata[j])
            tmpso2.append(soutdata[j])
            tmpci2.append(cindata[j])
            tmpco2.append(coutdata[j])
            tmpin2.append(intdata[j])
        #Previously home hours: >=22 or <=5.5
    
    #maxList= trsi + tmpsi + tmpsi2
    #print(max(maxList))
    #print(max(trsi+ tmpsi+ tmpsi2))
    #print("******")
    #print(max(tmpci))
    #print(max(tmpci2))
    #print("******")
    
    #With the following we make sure that we add the unique values of at most 1 of the values in a valid interval 
    #before we had cases in which values outside the intervals were added (bc they also had the same maxim value)
    for ik in (np.unique(dfs['time'].iloc[(dfs.smsin.values == max(tmpsi+ tmpsi2))])):
        dummyh =pd.to_datetime(ik)
        #print(dummyh)
        if(dummyh.hour+dummyh.minute/60 >= 10.5 and dummyh.hour+dummyh.minute/60 <= 12):
            smsipeak_wd.append(dummyh)
            break
        if(dummyh.hour+dummyh.minute/60 >=14 and dummyh.hour+dummyh.minute/60 <= 22):
            smsipeak_wd.append(dummyh)
            break
    #print("--------")
    
    for ik in (np.unique(dfs['time'].iloc[(dfs.smsout.values == max(tmpso+ tmpso2))])):
        dummyh =pd.to_datetime(ik)
        #print(dummyh)
        if(dummyh.hour+dummyh.minute/60 >= 10.5 and dummyh.hour+dummyh.minute/60 <= 12):
            smsopeak_wd.append(dummyh)
            break
        if(dummyh.hour+dummyh.minute/60 >=14 and dummyh.hour+dummyh.minute/60 <= 22):
            smsopeak_wd.append(dummyh)
            break
    #print("--------")
    
    #print(np.unique(dfs['time'].iloc[(dfs.callin.values == max(tmpci+ tmpci2))]))
    for ik in (np.unique(dfs['time'].iloc[(dfs.callin.values == max(tmpci+ tmpci2))])):
        dummyh =pd.to_datetime(ik)
        #print(dummyh)
        if(dummyh.hour+dummyh.minute/60 >= 10.5 and dummyh.hour+dummyh.minute/60 <= 12):
            callipeak_wd.append(dummyh)
            break
        if(dummyh.hour+dummyh.minute/60 >=14 and dummyh.hour+dummyh.minute/60 <= 22):
            callipeak_wd.append(dummyh)
            break
    #print("--------")
    
    for ik in (np.unique(dfs['time'].iloc[(dfs.callout.values == max(tmpco+ tmpco2))])):
        dummyh =pd.to_datetime(ik)
        #print(dummyh)
        if(dummyh.hour+dummyh.minute/60 >= 10.5 and dummyh.hour+dummyh.minute/60 <= 12):
            callopeak_wd.append(dummyh)
            break
        if(dummyh.hour+dummyh.minute/60 >=14 and dummyh.hour+dummyh.minute/60 <= 22):
            callopeak_wd.append(dummyh)
            break
    #print("--------")
    
    for ik in (np.unique(dfs['time'].iloc[(dfs.internet.values == max(tmpin+ tmpin2))])):
        dummyh =pd.to_datetime(ik)
        #print(dummyh)
        if(dummyh.hour+dummyh.minute/60 >= 10.5 and dummyh.hour+dummyh.minute/60 <= 12):
            intpeak_wd.append(dummyh)
            break
        if(dummyh.hour+dummyh.minute/60 >=14 and dummyh.hour+dummyh.minute/60 <= 22):
            intpeak_wd.append(dummyh)
            break
    #print("--------")
    
    #smsipeak_tr.append(dfs['time'].iloc[(dfs.smsin.values == max(trsi)).argmax()])
    #smsopeak_tr.append(dfs['time'].iloc[(dfs.smsout.values == max(trso)).argmax()])
    #callipeak_tr.append(dfs['time'].iloc[(dfs.callin.values == max(trci)).argmax()])
    #callopeak_tr.append(dfs['time'].iloc[(dfs.callout.values == max(trco)).argmax()])
    #intpeak_tr.append(dfs['time'].iloc[(dfs.internet.values == max(trin)).argmax()])
    
    if i==round(cells*0.10):
        print('10% completed')
    if i==round(cells*0.25):
        print('25% completed')
    if i==round(cells/2):
        print('50% completed')
    if i==round(cells*0.75):
        print('75% completed')
    if i==round(cells*0.9):
        print('90% completed')

#print(smsipeak_wd)
#print(callipeak_wd)

10% completed
25% completed
50% completed
75% completed
90% completed


In [5]:
pks_d = pd.DataFrame({}) #Weekday
pks_d=pks_d.append([smsipeak_wd, smsopeak_wd, callipeak_wd, callopeak_wd, intpeak_wd])

pks_d.to_csv("peaks_it_v31_wd.csv")

In [6]:
#print(callopeak_wd)
#print(callipeak_wd)
#print(intpeak_wd)
#print(len(smsipeak_wd))

#https://www.kaggle.com/andrewfager/analysis-and-modeling-of-internet-usage/notebook

In [7]:
#print(smsipeak_tr)
#print(intpeak_tr)